In [1]:
# !pip install -U torchdata
# !pip install -U spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !pip install portalocker
! pip install accelerate -U
! pip install evaluate
! pip install sentence-transformers
! pip install SentencePiece
! pip install bert_score
! pip install --upgrade nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
#  import T5
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import numpy as np
import os

import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    DebertaTokenizer,
    DebertaForSequenceClassification,
    AutoConfig,
)

import logging
import evaluate
from evaluate import load
from datasets import load_dataset

from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import T5Tokenizer,T5ForConditionalGeneration

import nltk
from nltk.translate.bleu_score import corpus_bleu

import matplotlib.pyplot as plt

from  transformers  import  AutoTokenizer, AutoModelWithLMHead

import csv
from torch.utils.data import DataLoader, Dataset
import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

device=""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'Using GPU: {torch.cuda.get_device_name()}')
else:
    device = torch.device("cpu")
    print('Using CPU')

2024-04-24 07:49:56.483720: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 07:49:56.483851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 07:49:56.605862: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using GPU: Tesla P100-PCIE-16GB


In [3]:
PATH="/kaggle/input/webis-clickbait-22/"
OUTPATH="/kaggle/working/"
# PATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\"
# OUTPATH="D:\\ghd\\NLP-Project\\webis-clickbait-22\\output\\"
BATCH=8
T5_model = "t5-base"
sep='[SEP]'

In [4]:
# data=pd.read_json(PATH+"train.jsonl",lines=True,encoding='utf-8')
# data.head()

In [5]:
def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')


    y = data["spoiler"]
    Y=[]
    for i in y:
        Y.append(" ".join(i))
        
    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    for i in dic:
        # print(i)
        i['postText']=i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']=targetparah
        # if(targetDescription is None or len(targetDescription)==0):
        #     targetDescription='none'
        #     desccount+=1
        # i['targetDescription']='targetDescription: '+targetDescription
        # targetkey=i['targetKeywords']
        # if(targetkey is None or len(targetkey)==0):
        #     targetkey='none'
        #     keycount+=1
        # i['targetKeywords']='targetKeywords: '+targetkey
        i['question']="question : "+i['postText']+"?"
        i['context']="context : "+i['targetParagraphs']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    # print("desccount: ",desccount)
    # print("keycount: ",keycount)
    X_q = [i['question'] for i in dic]
    X_q=np.array(X_q)
    X_c = [i['context'] for i in dic]
    X_c=np.array(X_c)

#     return X_q[:100],X_c[:100],Y[:100]
    return X_q,X_c,Y

In [6]:
x_train_q,x_train_c,y_train = get_split_data(PATH+'train.jsonl')
x_test_q,x_test_c,y_test = get_split_data(PATH+'test.jsonl',False)
x_val_q,x_val_c,y_val = get_split_data(PATH+'validation.jsonl',False)

In [7]:
x_train_q.shape,x_train_c.shape,len(y_train),x_test_q.shape,x_test_c.shape,len(y_test),x_val_q.shape,x_val_c.shape,len(y_val)

((3200,), (3200,), 3200, (1000,), (1000,), 1000, (800,), (800,), 800)

In [8]:
# create pd dataframe
train_df = pd.DataFrame(list(zip(x_train_q, x_train_c, y_train)), columns =['question', 'context', 'spoiler'])
val_df = pd.DataFrame(list(zip(x_val_q, x_val_c, y_val)), columns =['question', 'context', 'spoiler'])
test_df = pd.DataFrame(list(zip(x_test_q, x_test_c, y_test)), columns =['question', 'context', 'spoiler'])


In [9]:
train_df["question"][0],train_df["context"][0],train_df["spoiler"][0]

('question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we

In [10]:
# t5
tokenizer = AutoTokenizer.from_pretrained(T5_model)
model = AutoModelWithLMHead.from_pretrained(T5_model)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1682: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

### convert to Huggingface dataset
train_data = Dataset(pa.Table.from_pandas(train_df))
val_data = Dataset(pa.Table.from_pandas(val_df))
test_data = Dataset(pa.Table.from_pandas(test_df))
train_data,val_data,test_data

(Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 3200
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 800
 }),
 Dataset({
     features: ['question', 'context', 'spoiler'],
     num_rows: 1000
 }))

In [12]:

train_data[0]

{'question': 'question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?',
 'context': 'context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get 

In [13]:
def preproc(data):
    q=[]
    c=[]
    a=[]
    # print(len(data))
    # print(len(data['question']))
    # print(len(data['context']))
    # print(len(data['spoiler']))
    for i in range(len(data['question'])):
        q.append(data['question'][i])
        c.append(data['context'][i])
        a.append(data['spoiler'][i])

    model_inputs=tokenizer(q,c,text_target=a,return_tensors='pt',padding=True,truncation=True,max_length=512)
    return model_inputs


In [14]:
# train_data=Dataset.from_pandas(train_df)
tokenized_train=train_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_val=val_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_test=test_data.map(preproc,batched=True,batch_size=BATCH)
tokenized_train,tokenized_val,tokenized_test

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 3200
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 800
 }),
 Dataset({
     features: ['question', 'context', 'spoiler', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1000
 }))

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=T5_model)

In [16]:
import wandb
api_key = "9963cf6219e451d47251ea34645181ada1b2526b"
wandb.login(key=api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(T5_model)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [18]:
bertscore = load("bertscore")
meteor = evaluate.load("meteor")
bleu = evaluate.load("bleu")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [19]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    pred, ref = postprocess_text(decoded_preds, decoded_labels)

    # cal bleu, meteor and bert
    bleu_score = bleu.compute(predictions=pred, references=ref)
#     bleu1 = corpus_bleu(ref,pred,weights=(1,0,0,0))
#     bleu2 = corpus_bleu(ref,pred,weights=(0.5,0.5,0))
#     bleu3 = corpus_bleu(ref,pred,weights=(0.33,0.33,0.33,0)
#     bleu4 = corpus_bleu(ref,pred,weights=(0.25,0.25,0.25,0.25))

    print(bleu_score)
    meteor_score = meteor.compute(predictions=pred, references=ref)
    bertscore_score = bertscore.compute(predictions=pred, references=ref, lang="en")

    #  dict
    return {"blue":bleu_score["bleu"],
            "precisions_1":bleu_score["precisions"][0],
            "precisions_2":bleu_score["precisions"][1],
            "precisions_3":bleu_score["precisions"][2],
            "precisions_4":bleu_score["precisions"][3],
            "bp":bleu_score["brevity_penalty"],
            "meteor": meteor_score["meteor"], 
            "bertscore_f1": np.average(bertscore_score["f1"]), 
            "bertscore_p": np.average(bertscore_score["precision"]), 
            "bertscore_r": np.average(bertscore_score["recall"])}


In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./T5",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH,
    per_device_eval_batch_size=BATCH,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=12,
    predict_with_generate=True,
    save_strategy="epoch",
    fp16=True,
#     report_to="wandb",
#     logging_dir='./lolololol'

    # push_to_hub=True
    # load_best_model_at_end=True,
    # metric_for_best_model="bleu"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: raghav21274 (ragha). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240424_075107-km8u0jlr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exalted-pond-48
wandb: ⭐️ View project at https://wandb.ai/ragha/huggingface
wandb: 🚀 View run at https://wandb.ai/ragha/huggingface/runs/km8u0jlr


Epoch,Training Loss,Validation Loss,Blue,Precisions 1,Precisions 2,Precisions 3,Precisions 4,Bp,Meteor,Bertscore F1,Bertscore P,Bertscore R
1,No log,0.393570,0.073333,0.435517,0.345290,0.314754,0.299741,0.212482,0.291519,0.838177,0.844826,0.832971
2,1.203500,0.361358,0.086534,0.458090,0.365063,0.330918,0.314136,0.238314,0.329537,0.886916,0.895081,0.880190
3,0.328600,0.356305,0.100109,0.469305,0.378447,0.344374,0.326378,0.266331,0.350971,0.891155,0.899363,0.884259
4,0.313600,0.351624,0.105249,0.479539,0.386929,0.351629,0.333249,0.274088,0.366494,0.893402,0.901611,0.886452
5,0.286900,0.350660,0.103184,0.504180,0.419061,0.383981,0.366246,0.248528,0.370517,0.894552,0.903303,0.887089
6,0.286900,0.350326,0.111117,0.469968,0.384142,0.349182,0.331139,0.292331,0.369201,0.893383,0.900793,0.887150
7,0.274700,0.350387,0.111927,0.466517,0.378294,0.342367,0.323128,0.299429,0.371774,0.894219,0.902203,0.887421
8,0.255700,0.351656,0.107100,0.466405,0.377468,0.340121,0.320343,0.287784,0.371011,0.894090,0.901694,0.887668
9,0.261300,0.352131,0.104734,0.466447,0.376638,0.338593,0.318509,0.282296,0.369051,0.893769,0.901658,0.887090
10,0.248600,0.352672,0.107162,0.469223,0.383624,0.346372,0.327642,0.283456,0.366625,0.893716,0.901695,0.886937


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.07333300398744595, 'precisions': [0.4355167934681759, 0.3452895419187554, 0.31475409836065577, 0.2997408580477973], 'brevity_penalty': 0.21248244643240446, 'length_ratio': 0.39232673267326734, 'translation_length': 5389, 'reference_length': 13736}


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.11209835705589019, 'precisions': [0.466282236104437, 0.3775491456917141, 0.34016047297297297, 0.3206962327134001], 'brevity_penalty': 0.3011266763331528, 'length_ratio': 0.45449912638322654, 'translation_length': 6243, 'reference_length': 13736}


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.11233126726110775, 'precisions': [0.4703984513631231, 0.3847008705315799, 0.3473168654173765, 0.32795957651588065], 'brevity_penalty': 0.2964605075922226, 'length_ratio': 0.4512958648806057, 'translation_length': 6199, 'reference_length': 13736}


TrainOutput(global_step=4800, training_loss=0.3710923719406128, metrics={'train_runtime': 4870.3821, 'train_samples_per_second': 7.884, 'train_steps_per_second': 0.986, 'total_flos': 2.3383982997504e+16, 'train_loss': 0.3710923719406128, 'epoch': 12.0})

In [21]:
trainer.save_model("T5_overall")

In [22]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [23]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'bleu': 0.13912764173454834, 'precisions': [0.49973924380704043, 0.4017991004497751, 0.36227131515360717, 0.34160641000586284], 'brevity_penalty': 0.350417131451276, 'length_ratio': 0.48813084706930565, 'translation_length': 7670, 'reference_length': 15713}
{'eval_loss': 0.3466179072856903, 'eval_blue': 0.13912764173454834, 'eval_precisions_1': 0.49973924380704043, 'eval_precisions_2': 0.4017991004497751, 'eval_precisions_3': 0.36227131515360717, 'eval_precisions_4': 0.34160641000586284, 'eval_bp': 0.350417131451276, 'eval_meteor': 0.3937654021504598, 'eval_bertscore_f1': 0.897612532556057, 'eval_bertscore_p': 0.9051958106160164, 'eval_bertscore_r': 0.891330849468708, 'eval_runtime': 108.3963, 'eval_samples_per_second': 9.225, 'eval_steps_per_second': 1.153, 'epoch': 12.0}


In [24]:
def calculate_bleu(bp,precisions):
    weights_1 = np.array([1,0,0,0])
    weights_2 = np.array([0.5,0.5,0,0])
    weights_3 = np.array([0.33,0.33,0.33,0])
    weights_4 = np.array([0.25,0.25,0.25,0.25])
    logp = np.log(precisions)
        
    res1 = bp*np.exp(np.dot(logp,weights_1))
    res2 = bp*np.exp(np.dot(logp,weights_2))
    res3 = bp*np.exp(np.dot(logp,weights_3))
    res4 = bp*np.exp(np.dot(logp,weights_4))
    
    return [res1,res2,res3,res4]

In [25]:
results = trainer.evaluate(eval_dataset=tokenized_train)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.19113546972835826, 'precisions': [0.6484468577816817, 0.5801519729012176, 0.5522765598650927, 0.5380026249850853], 'brevity_penalty': 0.33056129094387526, 'length_ratio': 0.47461674025506434, 'translation_length': 25046, 'reference_length': 52771}
{'eval_loss': 0.20626115798950195, 'eval_blue': 0.19113546972835826, 'eval_precisions_1': 0.6484468577816817, 'eval_precisions_2': 0.5801519729012176, 'eval_precisions_3': 0.5522765598650927, 'eval_precisions_4': 0.5380026249850853, 'eval_bp': 0.33056129094387526, 'eval_meteor': 0.5003699872286638, 'eval_bertscore_f1': 0.921402530875057, 'eval_bertscore_p': 0.9306351635791361, 'eval_bertscore_r': 0.9133504011295736, 'eval_runtime': 344.0605, 'eval_samples_per_second': 9.301, 'eval_steps_per_second': 1.163, 'epoch': 12.0}


[0.21435143041681218,
 0.20274963343184824,
 0.19681174478800215,
 0.19113546972835826]

In [26]:
results = trainer.evaluate(eval_dataset=tokenized_val)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.11233126726110775, 'precisions': [0.4703984513631231, 0.3847008705315799, 0.3473168654173765, 0.32795957651588065], 'brevity_penalty': 0.2964605075922226, 'length_ratio': 0.4512958648806057, 'translation_length': 6199, 'reference_length': 13736}
{'eval_loss': 0.35329246520996094, 'eval_blue': 0.11233126726110775, 'eval_precisions_1': 0.4703984513631231, 'eval_precisions_2': 0.3847008705315799, 'eval_precisions_3': 0.3473168654173765, 'eval_precisions_4': 0.32795957651588065, 'eval_bp': 0.2964605075922226, 'eval_meteor': 0.3737784300359602, 'eval_bertscore_f1': 0.8948204495757818, 'eval_bertscore_p': 0.9023129215091467, 'eval_bertscore_r': 0.8884905031323433, 'eval_runtime': 87.3353, 'eval_samples_per_second': 9.16, 'eval_steps_per_second': 1.145, 'epoch': 12.0}


[0.13945456366170692,
 0.12611344055932877,
 0.11896268923721896,
 0.11233126726110775]

In [27]:
results = trainer.evaluate(eval_dataset=tokenized_test)
print(results)
calculate_bleu(results['eval_bp'],[results['eval_precisions_1'],results['eval_precisions_2'],results['eval_precisions_3'],results['eval_precisions_4']])

{'bleu': 0.13912764173454834, 'precisions': [0.49973924380704043, 0.4017991004497751, 0.36227131515360717, 0.34160641000586284], 'brevity_penalty': 0.350417131451276, 'length_ratio': 0.48813084706930565, 'translation_length': 7670, 'reference_length': 15713}
{'eval_loss': 0.3466179072856903, 'eval_blue': 0.13912764173454834, 'eval_precisions_1': 0.49973924380704043, 'eval_precisions_2': 0.4017991004497751, 'eval_precisions_3': 0.36227131515360717, 'eval_precisions_4': 0.34160641000586284, 'eval_bp': 0.350417131451276, 'eval_meteor': 0.3937654021504598, 'eval_bertscore_f1': 0.897612532556057, 'eval_bertscore_p': 0.9051958106160164, 'eval_bertscore_r': 0.891330849468708, 'eval_runtime': 107.6347, 'eval_samples_per_second': 9.291, 'eval_steps_per_second': 1.161, 'epoch': 12.0}


[0.17511719228849296,
 0.15702237353733228,
 0.1475621009464393,
 0.1391276417345483]

In [28]:
q='question : Wes Welker Wanted Dinner With Tom Brady, But Patriots QB Had Better Idea?'
c='context : It’ll be just like old times this weekend for Tom Brady and Wes Welker. Welker revealed Friday morning on a Miami radio station that he contacted Brady because he’ll be in town for Sunday’s game between the New England Patriots and Miami Dolphins at Gillette Stadium. It seemed like a perfect opportunity for the two to catch up. But Brady’s definition of "catching up" involves far more than just a meal. In fact, it involves some literal "catching" as the Patriots quarterback looks to stay sharp during his four-game Deflategate suspension. "I hit him up to do dinner Saturday night. He’s like, ‘I’m going to be flying in from Ann Arbor later (after the Michigan-Colorado football game), but how about that morning we go throw?’ " Welker said on WQAM, per The Boston Globe. "And I’m just sitting there, I’m like, ‘I was just thinking about dinner, but yeah, sure. I’ll get over there early and we can throw a little bit.’ " Welker was one of Brady’s favorite targets for six seasons from 2007 to 2012. It’s understandable him and Brady want to meet with both being in the same area. But Brady typically is all business during football season. Welker probably should have known what he was getting into when reaching out to his buddy. "That’s the only thing we really have planned," Welker said of his upcoming workout with Brady. "It’s just funny. I’m sitting there trying to have dinner. ‘Hey, get your ass up here and let’s go throw.’ I’m like, ‘Aw jeez, man.’ He’s going to have me running like 2-minute drills in his backyard or something." Maybe Brady will put a good word in for Welker down in Foxboro if the former Patriots wide receiver impresses him enough.'
enc=tokenizer(q+" "+c,return_tensors='pt',padding=True,truncation=True,max_length=512)
out=model.generate(input_ids=enc['input_ids'].to(device))

In [29]:
tokenizer.decode(out[0]),'how about that morning we go throw?'

('<pad> throw</s>', 'how about that morning we go throw?')

In [30]:
import os
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/working/__notebook__.ipynb
/kaggle/working/T5_overall/tokenizer.json
/kaggle/working/T5_overall/special_tokens_map.json
/kaggle/working/T5_overall/training_args.bin
/kaggle/working/T5_overall/generation_config.json
/kaggle/working/T5_overall/model.safetensors
/kaggle/working/T5_overall/config.json
/kaggle/working/T5_overall/tokenizer_config.json
/kaggle/working/T5_overall/spiece.model
/kaggle/working/wandb/debug.log
/kaggle/working/wandb/debug-internal.log
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/run-km8u0jlr.wandb
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/wandb-metadata.json
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/conda-environment.yaml
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/output.log
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/config.yaml
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/wandb-summary.json
/kaggle/working/wandb/run-20240424_075107-km8u0jlr/files/requirements.txt
/kaggle/working/w